# Stellar Origin

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import json

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse

In [3]:
figure_setup()
settings = Settings()

In [4]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [5]:
def read_data(simulation: str) -> pd.DataFrame:
    settings = Settings()
    
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)
    s.tag_in_situ_stars()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame({
        "RegionTag": s.region_tag[is_real_star & is_main_obj],
        "IsInSitu": s.is_in_situ[is_real_star & is_main_obj],
        })

    return df

In [6]:
def prepare_data(df: pd.DataFrame) -> tuple:
    """
    This method takes a data frame with stellar information and returns a
    tuple with the ordered values of in-situ stellar percentages, labels and
    colors.

    Returns
    -------
    tuple
        The prepared data.
    """

    settings = Settings()

    # Bar widths
    bar_widths = np.zeros(len(settings.components) + 1)
    bar_widths[0] = df["IsInSitu"].sum() / len(df)  # Galaxy
    for i in range(len(settings.components)):
        bar_widths[i + 1] = (df["IsInSitu"][df["RegionTag"] == i]).sum() \
            / len(df[df["RegionTag"] == i])

    # Numerical labels
    num_labels = np.array([
        r"$\textbf{" + str(np.round(100 * item, 1)) \
            + r"\%" "}$" for item in bar_widths])

    # Categorical labels
    labels = np.array(
        ["Galaxy"] + [
        item.replace(" ", "\n") for item in \
            settings.component_labels.values()])

    # Colors
    colors = np.array(["tab:gray"] + list(settings.component_colors.values()))

    order_idx = np.argsort(bar_widths)  # Order data

    return (bar_widths[order_idx], num_labels[order_idx],
        labels[order_idx], colors[order_idx])

In [9]:
data = {}
for galaxy in settings.groups["Included"]:
    simulation = f"au{galaxy}_or_l4_s127"
    df = read_data(simulation)
    bar_widths, num_labels, labels, colors = prepare_data(df)

    data[simulation] = {
        "BarWidths": list(bar_widths),
        "NumericalLabels": list(num_labels),
        "ComponentLabel": list(labels),
        "Colors": list(colors),
        }

    # Sava data as json
    with open(f"../results/stellar_origin{SUFFIX}.json", "w") as f:
        json.dump(data, f)

In [24]:
def transform_label(label: str) -> str:
    if label == "Halo":
        new_label = 'H'
    elif label == "Bulge":
        new_label = 'B'
    elif label == "Warm\nDisc":
        new_label = 'WD'
    elif label == "Cold\nDisc":
        new_label = 'CD'
    else:
        new_label = "Galaxy"
    return new_label

In [127]:
def get_min_and_max_values(sample: list) -> tuple:
    settings = Settings()

    with open(f"../results/stellar_origin{SUFFIX}.json") as f:
        data = json.load(f)

    COMPONENTS = ['Halo', 'Cold\nDisc', 'Warm\nDisc', 'Galaxy', 'Bulge']

    extreme_vals = []
    for component in COMPONENTS:
        max_val = 0
        min_val = 1
        for simulation in sample:
            idx = data[simulation]["ComponentLabel"].index(component)
            if data[simulation]["BarWidths"][idx] < min_val:
                min_val = data[simulation]["BarWidths"][idx]
            if data[simulation]["BarWidths"][idx] > max_val:
                max_val = data[simulation]["BarWidths"][idx]
        extreme_vals.append([min_val, max_val])

    return extreme_vals

In [159]:
SAMPLE = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]

for SIMULATION in SAMPLE:

    # Read data
    with open(f"../results/stellar_origin{SUFFIX}.json") as f:
        data = json.load(f)[SIMULATION]
        bar_widths = np.array(data["BarWidths"])
        num_labels = data["NumericalLabels"]
        labels = [transform_label(label) for label in \
            data["ComponentLabel"]]
        colors = data["Colors"]

    galaxy = f"Au{parse(SIMULATION)[0]}"

    if SIMULATION == "au6_or_l4_s127":
        extreme_values = np.array(get_min_and_max_values(SAMPLE))
        yerr = 100 * np.abs(
            np.array(
                data["BarWidths"]).reshape(5, 1) - extreme_values).T
    else:
        yerr = None

    fig = plt.figure(figsize=(3.0, 2.25))
    gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
    ax = gs.subplots(sharex=True, sharey=True)

    ax.label_outer()
    ax.tick_params(which='both', direction="in",
                    bottom=False, top=False, left=False, right=False)
    ax.set_xlim(0, 100)
    ax.set_xticks([0, 20, 40, 60, 80, 100])
    ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_ylim(-0.5, 4.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

    hbars = ax.barh(y=np.arange(bar_widths.shape[0]), xerr=yerr,
                    width=100 * bar_widths, color=colors, linewidth=0)
    padding = 1 if 100 * bar_widths.min() <= 20 else -30
    ax.set_yticks([])
    for i, label in enumerate(labels):
        ax.text(x=2, y=i, size=8.0, ha="left", va="center",
            s=r"$\textbf{" + num_labels[i] + "}$", c="white")
        ax.text(
            x=-2, y=i, size=8.0, ha="right", va="center",
            s=r"$\textbf{" + label + "}$",
            c=colors[i])
    ax.set_xlabel(r"$f_\mathrm{in-situ}$")

    ax.text(x=ax.get_xlim()[0], y=ax.get_ylim()[1],
            s=r"$\texttt{" + galaxy + "}$",
            size=9.0, ha='left', va='bottom',
            )

    fig.savefig(f"../images/stellar_origin_by_region/{SIMULATION}{SUFFIX}.pdf")
    plt.close(fig)